In [ ]:
# SGR 0418+5729 fitting
# HS1 belt, HS2 with limited size
#　一個自由なホットスポットと回転軸に対して対称性のあるベルト


import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import numpy as np
import math
import scipy.optimize
import os
import matplotlib
from mpl_toolkits.mplot3d import axes3d, Axes3D


# load data
# magnetar SGR 0418+5729 from paper:
df = pd.read_csv('SGR 0418+5729_first(blue)_obs.csv', header = 0)
x_data = df.iloc[:,0]
y_data = df.iloc[:,1]
N_data = len(x_data)
if N_data % 2 == 0:
    print("Correct dataset")
else:
    print("dataset error")

# geometric figure
# Create a new figure
fig = plt.figure(figsize = (12.0, 6.0), dpi = 100)
ax2 = plt.subplot2grid((20, 11), (2, 5), rowspan = 18, colspan = 5, projection = "3d")
ax1 = plt.subplot2grid((20, 11), (2, 0), rowspan = 11, colspan = 5)
ax3 = plt.subplot2grid((20, 11), (14, 0), rowspan = 4, colspan = 5)
# Get the current axes, and tell Matplotlib to use the 3D projection

# Set the scaling factor for the sphere
scale = 1.01

# Create a sphere
r = 1 # radius
phi_sphere, theta_sphere = np.mgrid[0:np.pi:100j, 0:2 * np.pi:100j]
x_sphere = r * np.sin(phi_sphere) * np.cos(theta_sphere)
y_sphere = r * np.sin(phi_sphere) * np.sin(theta_sphere)
z_sphere = r * np.cos(phi_sphere)

# Plot the sphere
ax2.plot_surface(x_sphere, y_sphere, z_sphere, alpha = 0.3, color = "aquamarine", edgecolor = "w", linewidth = 0.1)



# pulse profile
# theta = pi / 5
# i = pi / 3
# a = theta, b = i, c = phi, d = normalization
# initial parameter setting
i_ini = np.radians(35)
# HS1
theta_1_main_ini = np.radians(128)
phi_1_main_ini = np.radians(110)
m_1_main_ini = 3.4
ang_1_sub_ini = np.radians(10)
m_1_sub_ini = 0.8

# HS2
theta_2_main_ini = np.radians(145)
phi_2_main_ini = np.radians(250)
m_2_main_ini = 7
ang_2_sub_ini = np.radians(10)
m_2_sub_ini = 2

# light-bending factor
f_lb = 1. / 3.

# sub_hot_spot_number
N_sub = 10
N_belt = 4

#############################################################################################
# function area

def mu(x, i, theta, phi, m):
    y = np.sin(theta) * np.sin(i) * np.cos(2 * math.pi * x - phi) + np.cos(theta) * np.cos(i)
    F = m * (y * (1. - f_lb) + f_lb)
    result = (abs(F) + F) / 2
    return result

def rectangular_to_polar_theta(z):
    theta_polar = np.arccos(z / (r * scale))
    return theta_polar

def rectangular_to_polar_phi(x, y):
    phi_polar = np.arctan(y / x)
    return phi_polar

def polar_to_rectangular_x(theta, phi):
    x_rectangular = scale * r * np.sin(theta) * np.cos(phi)
    return x_rectangular

def polar_to_rectangular_y(theta, phi):
    y_rectangular = scale * r * np.sin(theta) * np.sin(phi)
    return y_rectangular

def polar_to_rectangular_z(theta):
    z_rectangular = scale * r * np.cos(theta)
    return z_rectangular

def rotation_matrix(theta, phi, x, y, z):
    xyz = np.array([[x], [y], [z]])
    z_rot = np.array([[np.cos(-phi), np.sin(-phi), 0], [-np.sin(-phi), np.cos(-phi), 0], [0, 0 ,1]])
    y_rot = np.array([[np.cos(-theta), 0, -np.sin(-theta)], [0, 1, 0], [np.sin(-theta), 0, np.cos(-theta)]])
    XYZ_after_rotation = np.dot(z_rot, np.dot(y_rot, xyz))
    return XYZ_after_rotation

def sub_hot_spot_hs(N_sub, ang, theta_main, phi_main):
    delta_phi = 360 / N_sub
    temp_phi = 0
    x_hs_sub = np.empty(0)
    y_hs_sub = np.empty(0)
    z_hs_sub = np.empty(0)
    for i in range(0, N_sub):
        x_hs_sub = np.append(x_hs_sub, polar_to_rectangular_x(ang, np.radians(temp_phi)))
        y_hs_sub = np.append(y_hs_sub, polar_to_rectangular_y(ang, np.radians(temp_phi)))
        z_hs_sub = np.append(z_hs_sub, polar_to_rectangular_z(ang))
        temp_phi += delta_phi
    hs_sub_XYZ = np.empty((3, 1))
    for i in range(0, N_sub):
        hs_sub_XYZ = np.append(hs_sub_XYZ, rotation_matrix(theta_main, phi_main, x_hs_sub[i], y_hs_sub[i], z_hs_sub[i]), axis = 1)
    hs_sub_theta = np.empty(0)
    hs_sub_phi = np.empty(0)
    for i in range(0, N_sub):
        hs_sub_theta = np.append(hs_sub_theta, rectangular_to_polar_theta(hs_sub_XYZ[2, i]))
        hs_sub_phi = np.append(hs_sub_phi, rectangular_to_polar_phi(hs_sub_XYZ[1, i], hs_sub_XYZ[0, i]))
    return hs_sub_XYZ, hs_sub_theta, hs_sub_phi

def belt_like(N_belt, ang, theta_main, phi_main):
    phi_belt = np.empty(0)
    temp_phi = phi_main - N_belt / 2 * ang
    k = 1
    for i in range(0, N_belt):
        phi_belt = np.append(phi_belt, temp_phi)
        if k == N_belt / 2:
            temp_phi += ang
        temp_phi += ang
        k += 1
    x_hs_sub = np.empty(0)
    y_hs_sub = np.empty(0)
    z_hs_sub = np.empty(0)
    for i in range(0, N_sub):
        x_hs_sub = np.append(x_hs_sub, polar_to_rectangular_x(theta_main, np.radians(phi_belt[i])))
        y_hs_sub = np.append(y_hs_sub, polar_to_rectangular_y(theta_main, np.radians(phi_belt[i])))
        z_hs_sub = np.append(z_hs_sub, polar_to_rectangular_z(theta_main))
    return phi_belt, x_hs_sub, y_hs_sub, z_hs_sub

#############################################################################################
# limited size hot spot
def HS1(x, i, theta_1_main_ini, phi_1_main_ini, m_1_main_ini, ang_1_sub_ini, m_1_sub_ini):
    # HS1 sub
    hs_1_sub_XYZ, hs_1_sub_theta, hs_1_sub_phi = sub_hot_spot_hs(N_sub, ang_1_sub_ini, theta_1_main_ini, phi_1_main_ini)
    HS1 = mu(x, i, theta_1_main_ini, phi_1_main_ini, m_1_main_ini)
    for i in range(0, N_sub):
        HS1 += mu(x, i, hs_1_sub_theta[i], hs_1_sub_phi[i], m_1_sub_ini)
    return HS1

# belt like hot spot
def HS2(x, i, theta_2_main_ini, phi_2_main_ini, m_2_main_ini, ang_2_sub_ini, m_2_sub_ini):
    # HS2 sub
    hs_2_sub_XYZ, hs_2_sub_theta, hs_2_sub_phi = sub_hot_spot_hs(N_sub, ang_2_sub_ini, theta_2_main_ini, phi_2_main_ini)
    HS2 = mu(x, i, theta_2_main_ini, phi_2_main_ini, m_2_main_ini)
    for i in range(0, N_belt):
        HS2 += mu(x, i, hs_2_sub_theta[i], hs_2_sub_phi[i], m_2_sub_ini)
    return HS2


def mu_total(x, i, theta_1_main_ini, phi_1_main_ini, m_1_main_ini, ang_1_sub_ini, m_1_sub_ini, theta_2_main_ini, phi_2_main_ini, m_2_main_ini, ang_2_sub_ini, m_2_sub_ini):
    # HS1 sub
    hs_1_sub_XYZ, hs_1_sub_theta, hs_1_sub_phi = sub_hot_spot_hs(N_sub, ang_1_sub_ini, theta_1_main_ini, phi_1_main_ini)
    # HS2 sub
    hs_2_sub_XYZ, hs_2_sub_theta, hs_2_sub_phi = sub_hot_spot_hs(N_sub, ang_2_sub_ini, theta_2_main_ini, phi_2_main_ini)
    # total
    Total = mu(x, i, theta_1_main_ini, phi_1_main_ini, m_1_main_ini) + \
            mu(x, i, theta_2_main_ini, phi_2_main_ini, m_2_main_ini)
    for i in range(0, N_sub):
        Total += mu(x, i, hs_1_sub_theta[i], hs_1_sub_phi[i], m_1_sub_ini)
        Total += mu(x, i, hs_2_sub_theta[i], hs_2_sub_phi[i], m_2_sub_ini)
    return Total

x = np.linspace(0, 2, 5000)

x_data_fit = x_data[0:N_data/2]
y_data_fit = y_data[0:N_data/2]


# fitting
# parameter_initial = np.array([-1.5, 1, 0.2, 4000])
# parameter_initial_2 = np.array([-1.5, 1, 0.2, 4000])
parameter_initial_3 = np.array([i_ini, theta_1_main_ini, phi_1_main_ini, m_1_main_ini, ang_1_sub_ini, m_1_sub_ini, theta_2_main_ini, phi_2_main_ini, m_2_main_ini, ang_2_sub_ini, m_2_sub_ini])

# parameter_initial_4 = np.array([-1.5, 1, 0.2, 4000])

# parameter_optimal, covariance = scipy.optimize.curve_fit(func_3, x_data, y_data, p0 = parameter_initial_3)


# parameter_optimal, covariance = scipy.optimize.curve_fit(mu_total, x_data, y_data, p0 = parameter_initial_3, \
#                                                         bounds = ((0, 0, 0, 0, 7, 0, 0, 0, 0, 7, 0), \
#                                                                   (90, 360, 180, 100, 20.5, 10, 90, 360, 100, 20.5, 10)))

parameter_optimal, covariance = scipy.optimize.curve_fit(mu_total, x_data_fit, y_data_fit, p0 = parameter_initial_3)

# print(parameter_optimal)

# y_fitting = func_3(x, parameter_optimal[0], parameter_optimal[1], parameter_optimal[2], parameter_optimal[3], parameter_optimal[4], parameter_optimal[5], parameter_optimal[6], parameter_optimal[7], parameter_optimal[8], parameter_optimal[9])
# y_fitting_same_size_as_y_data = func_3(x_data, parameter_optimal[0], parameter_optimal[1], parameter_optimal[2], parameter_optimal[3], parameter_optimal[4], parameter_optimal[5], parameter_optimal[6], parameter_optimal[7], parameter_optimal[8], parameter_optimal[9])
y_fitting = mu_total(x, parameter_optimal[0], parameter_optimal[1], parameter_optimal[2], parameter_optimal[3], parameter_optimal[4], parameter_optimal[5], parameter_optimal[6], parameter_optimal[7], parameter_optimal[8], parameter_optimal[9], parameter_optimal[10])
y_fitting_same_size_as_y_data = mu_total(x_data, parameter_optimal[0], parameter_optimal[1], parameter_optimal[2], parameter_optimal[3], parameter_optimal[4], parameter_optimal[5], parameter_optimal[6], parameter_optimal[7], parameter_optimal[8], parameter_optimal[9], parameter_optimal[10])

# trajectory of HS
x_track_hs_1 = scale * r * np.sin(parameter_optimal[1]) * np.cos(theta_sphere)
y_track_hs_1 = scale * r * np.sin(parameter_optimal[1]) * np.sin(theta_sphere)
z_track_hs_1 = scale * r * np.cos(parameter_optimal[1])
x_track_hs_2 = scale * r * np.sin(parameter_optimal[6]) * np.cos(theta_sphere)
y_track_hs_2 = scale * r * np.sin(parameter_optimal[6]) * np.sin(theta_sphere)
z_track_hs_2 = scale * r * np.cos(parameter_optimal[6])
ax2.scatter(x_track_hs_1, y_track_hs_1, z_track_hs_1, s = 3, color = "silver")
ax2.scatter(x_track_hs_2, y_track_hs_2, z_track_hs_2, s = 3, color = "silver")

# positions of HS
x_hs_1 = scale * r * np.sin(parameter_optimal[1]) * np.cos(parameter_optimal[2])
x_hs_2 = scale * r * np.sin(parameter_optimal[6]) * np.cos(parameter_optimal[7])
y_hs_1 = scale * r * np.sin(parameter_optimal[1]) * np.sin(parameter_optimal[2])
y_hs_2 = scale * r * np.sin(parameter_optimal[6]) * np.sin(parameter_optimal[7])
z_hs_1 = scale * r * np.cos(parameter_optimal[1])
z_hs_2 = scale * r * np.cos(parameter_optimal[6])

x_hs = [x_hs_1, x_hs_2]
y_hs = [y_hs_1, y_hs_2]
z_hs = [z_hs_1, z_hs_2]

HS_main_plot_size = 30
HS_sub_plot_size_1 = m_1_sub_ini / m_1_main_ini * HS_main_plot_size
HS_sub_plot_size_2 = m_2_sub_ini / m_2_main_ini * HS_main_plot_size

ax2.scatter(x_hs[0], y_hs[0], z_hs[0], s = HS_main_plot_size, c = "blue")
ax2.scatter(x_hs[1], y_hs[1], z_hs[1], s = HS_main_plot_size, c = "orange")



# positions of HS_sub

# maybe useless, for drawing subplot
def drawing_sub_hs(theta_sub_fit, N_sub, theta_main_fit, phi_main_fit):
    x_draw = np.empty(0)
    y_draw = np.empty(0)
    z_draw = np.empty(0)
    for i in range(0, N_sub):
        x_draw = np.append(x_draw, polar_to_rectangular_x(theta_sub_fit, np.radians(i * 360 / N_sub)))
        y_draw = np.append(y_draw, polar_to_rectangular_y(theta_sub_fit, np.radians(i * 360 / N_sub)))
        z_draw = np.append(z_draw, polar_to_rectangular_z(theta_sub_fit))
    hs_sub_draw = np.empty((3, 1))
    for i in range(0, N_sub):
        hs_sub_draw = np.append(hs_sub_draw, rotation_matrix(theta_main_fit, phi_main_fit, x_draw[i], y_draw[i], z_draw[i]), axis = 1)
    return hs_sub_draw

hs_sub_draw_1 = drawing_sub_hs(parameter_optimal[4], N_sub, parameter_optimal[1], parameter_optimal[2])
print(hs_sub_draw_1)
for i in range(0, N_sub):
    ax2.scatter(hs_sub_draw_1[0, i + 1], hs_sub_draw_1[1, i + 1], hs_sub_draw_1[2, i + 1], zorder = 3, s = HS_sub_plot_size_1, c = "lime")

hs_sub_draw_2 = drawing_sub_hs(parameter_optimal[9], N_sub, parameter_optimal[6], parameter_optimal[7])
for i in range(0, N_sub):
    ax2.scatter(hs_sub_draw_2[0, i + 1], hs_sub_draw_2[1, i + 1], hs_sub_draw_2[2, i + 1], zorder = 3, s = HS_sub_plot_size_2, c = "green")



HS1 = HS1(x, parameter_optimal[0], parameter_optimal[1], parameter_optimal[2], parameter_optimal[3], parameter_optimal[4], parameter_optimal[5])
HS2 = HS2(x, parameter_optimal[0], parameter_optimal[6], parameter_optimal[7], parameter_optimal[8], parameter_optimal[9], parameter_optimal[10])



# data of pulse profile
ax1.scatter(x_data, y_data, s = 5, c = "dimgrey")
# fitting of pulse profile
ax1.plot(x, HS1, label = "HS1", color = "blue")
ax1.plot(x, HS2, label = "HS2", color = "orange")
ax1.plot(x, y_fitting, label = 'total', color = 'lime')
# info
ax1.set_xlim(0, 2)
ax1.set_xlabel('Phase')
ax1.set_ylabel('Counts/sec')
ax1.legend(bbox_to_anchor = (1.15, 1), loc = "upper right")
ax1.set_title("SGR 0418+5729, 0.5-10 keV, light-bending model with stationary emission")
# ax.text(1.8, 0.325, r"$\theta = \frac{\pi}{5}$")
# ax.text(1.8, 0.31, r"$i = \frac{\pi}{3}$")
# ax.text(1.8, 0.295, r"$\phi = \pi$")
# ax.text(0.8, 0.32, r"$amp = 0.345$")
# plt.title("3XMM J185246.6+003317のパルス波形と手合わせ曲線", fontname = "MS Gothic")

ax2.set_xlim(-1.1, 1.1)
ax2.set_ylim(-1.1, 1.1)
ax2.set_zlim(-1.1, 1.1)
ax2.view_init(elev = np.degrees(parameter_optimal[0]), azim = 0)
ax2.axis("off")

# ax3.plot(x_data, y_data - func(x_data, a_p_1, b, c_p_1, d_p_1) - func(x_data, a_p_2, b, c_p_2, d_p_2) - func(x_data, a_p_3, b, c_p_3, d_p_3))
ax3.plot(x_data, (y_data - y_fitting_same_size_as_y_data) / y_data)
ax3.axhline(y = 0, linestyle = '--', color = 'grey')
ax3.set_xlim(0, 2)
ax3.set_ylabel('residual/data')
plt.show()
print("inclination angle:", np.degrees(parameter_optimal[0]))
print("Hot_spot_1:", np.degrees(parameter_optimal[1]), np.degrees(parameter_optimal[2]), parameter_optimal[3])
print("Hot_spot_1_sub:", np.degrees(parameter_optimal[4]), parameter_optimal[5])
print("Hot_spot_2:", np.degrees(parameter_optimal[6]), np.degrees(parameter_optimal[7]), parameter_optimal[8])
print("Hot_spot_1_sub:", np.degrees(parameter_optimal[9]), parameter_optimal[10])

In [1]:
# SGR 0418+5729 fitting
# HS1&2 on magnetic polar, with belt in magnetical equatorial


import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import numpy as np
import math
import scipy.optimize
import os
import matplotlib
from mpl_toolkits.mplot3d import axes3d, Axes3D


# load data
# magnetar SGR 0418+5729 from paper:
df = pd.read_csv('SGR 0418+5729_first(blue)_obs.csv', header = 0)
x_data = df.iloc[:,0]
y_data = df.iloc[:,1]
lenth_x_data = len(x_data)
lenth_y_data = len(y_data)
N_data = len(x_data)
if N_data % 2 == 0:
    print("Correct dataset")
else:
    print("dataset error")


# geometric figure
# Create a new figure
fig = plt.figure(figsize = (12.0, 6.0), dpi = 100)
ax2 = plt.subplot2grid((20, 11), (2, 5), rowspan = 18, colspan = 5, projection = "3d")
ax1 = plt.subplot2grid((20, 11), (2, 0), rowspan = 11, colspan = 5)
ax3 = plt.subplot2grid((20, 11), (14, 0), rowspan = 4, colspan = 5)
# Get the current axes, and tell Matplotlib to use the 3D projection

# Set the scaling factor for the sphere
scale = 1.01

# Create a sphere
r = 1 # radius
phi_sphere, theta_sphere = np.mgrid[0:np.pi:100j, 0:2 * np.pi:100j]
x_sphere = r * np.sin(phi_sphere) * np.cos(theta_sphere)
y_sphere = r * np.sin(phi_sphere) * np.sin(theta_sphere)
z_sphere = r * np.cos(phi_sphere)

# Plot the sphere
ax2.plot_surface(x_sphere, y_sphere, z_sphere, alpha = 0.3, color = "aquamarine", edgecolor = "w", linewidth = 0.1)



# pulse profile
# initial parameter setup
i_ini = np.radians(35)

# HS
theta_main_ini = np.radians(128)
phi_main_ini = np.radians(110)
m_main_ini = 3.4

# belt on equator
equator = np.radians(90)

# belt_like
#theta_belt_center_ini = np.radians(50)
theta_belt_center = equator
phi_belt_center_ini = np.radians(140)
ang_belt_ini = np.radians(5)
m_belt_ini = 2

# light-bending factor
f_lb = 1. / 3.

# sub_hot_spot_number
N_sub = 10
N_belt = 4

# belt on equator
equator = np.radians(90)

#############################################################################################
# function area

def mu(x, i, theta, phi, m):
    y = np.sin(theta) * np.sin(i) * np.cos(2 * math.pi * x - phi) + np.cos(theta) * np.cos(i)
    F = m * (y * (1. - f_lb) + f_lb)
    result = (abs(F) + F) / 2
    return result

def rectangular_to_polar_theta(z):
    theta_polar = np.arccos(z / (r * scale))
    return theta_polar

def rectangular_to_polar_phi(x, y):
    phi_polar = np.arctan(y / x)
    return phi_polar

def polar_to_rectangular_x(theta, phi):
    x_rectangular = scale * r * np.sin(theta) * np.cos(phi)
    return x_rectangular

def polar_to_rectangular_y(theta, phi):
    y_rectangular = scale * r * np.sin(theta) * np.sin(phi)
    return y_rectangular

def polar_to_rectangular_z(theta):
    z_rectangular = scale * r * np.cos(theta)
    return z_rectangular

def rotation_matrix(theta, phi, x, y, z):
    xyz = np.array([[x], [y], [z]])
    z_rot = np.array([[np.cos(-phi), np.sin(-phi), 0], [-np.sin(-phi), np.cos(-phi), 0], [0, 0 ,1]])
    y_rot = np.array([[np.cos(-theta), 0, -np.sin(-theta)], [0, 1, 0], [np.sin(-theta), 0, np.cos(-theta)]])
    XYZ_after_rotation = np.dot(z_rot, np.dot(y_rot, xyz))
    return XYZ_after_rotation

# geometry of belt
# phi_belt as delta phi
def belt_like(N_belt, ang, theta_center, phi_center):
    phi_belt_xyz = np.empty(0)
    temp_phi_xyz = phi_center
    for i in range(0, N_belt):
        phi_belt_xyz = np.append(phi_belt_xyz, temp_phi_xyz)
        temp_phi_xyz += ang
    x_belt = np.empty(0)
    y_belt = np.empty(0)
    z_belt = np.empty(0)
    for i in range(0, N_sub):
        x_belt = np.append(x_belt, polar_to_rectangular_x(equator, np.radians(phi_belt[i])))
        y_belt = np.append(y_belt, polar_to_rectangular_y(equator, np.radians(phi_belt[i])))
        z_belt = np.append(z_belt, polar_to_rectangular_z(equator))
    belt_XYZ = np.empty((3, 1))
    for i in range(0, N_sub):
        belt_XYZ = np.append(belt_XYZ, rotation_matrix(theta_center, phi_center, x_belt, y_belt, z_belt))
    belt_theta = np.empty(0)
    belt_phi = np.empty(0)
    for i in range(0, N_sub):
        belt_theta = np.append(belt_theta, rectangular_to_polar_theta(belt_XYZ[2, i]))
        belt_phi = np.append(belt_phi, rectangular_to_polar_phi(belt_XYZ[1, i], belt_XYZ[0, i]))
    return belt_XYZ, belt_theta, belt_phi

######## need to be recode ########
def position_of_hot_spot(theta_main, phi_main):
    belt_XYZ, belt_theta, belt_phi = belt_like()

# useless function
def calculate_reduced_chi_square(data, model, degree_of_freedom):
    RCS = scipy.stats.chisquare(data, model, degree_of_freedom)
    return RCS

#############################################################################################

# pulse from belt
def belt_mu(N_belt, ang_belt, phi_belt, theta_center, phi_center, m_belt):
    # belt like bbody
    belt_XYZ, belt_theta, belt_phi = belt_like(N_belt, ang_belt, phi_belt, theta_1_main, phi_1_main)
    belt_mu = 0
    for i in range(0, N_sub):
        belt_mu += mu(x, i_ini, belt_theta[i], belt_phi[i], m_belt)
    return belt_mu

# total
def mu_total(x, i, theta_1_main, phi_1_main, m_1_main, ang_belt, m_belt):
    # dot resource hot spot
    HS1 = mu(x, i, theta_1_main, phi_1_main, m_1_main)
    # dot resource hot spot
    belt = belt_mu(N_belt, ang_belt, phi)
    #total
    Total = HS1 + HS2 + belt_mu(N_belt, ang_belt, theta_1_main, phi_1_main, m_belt)
    return Total

x = np.linspace(0, 2, 5000)

x_data_fit = x_data[0:N_data/2]
y_data_fit = y_data[0:N_data/2]

# fitting
# initial setup list for cuve_fit
parameter_initial_3 = np.array([i_ini, theta_1_main_ini, phi_1_main_ini, m_1_main_ini, m_2_main_ini, ang_belt_ini, phi_belt_ini])

# single period data for fitting
parameter_optimal, covariance = scipy.optimize.curve_fit(mu_total, x_data_fit, y_data_fit, p0 = parameter_initial_3)

# double periods for figure
y_fitting = mu_total(x, parameter_optimal[0], parameter_optimal[1], parameter_optimal[2], parameter_optimal[3], parameter_optimal[4], parameter_optimal[5], parameter_optimal[6])
y_fitting_same_size_as_y_data = mu_total(x_data, parameter_optimal[0], parameter_optimal[1], parameter_optimal[2], parameter_optimal[3], parameter_optimal[4], parameter_optimal[5], parameter_optimal[6])


######## need to be recode ########
# positions of HS
x_hs_1 = scale * r * np.sin(parameter_optimal[1]) * np.cos(parameter_optimal[2])
y_hs_1 = scale * r * np.sin(parameter_optimal[1]) * np.sin(parameter_optimal[2])
z_hs_1 = scale * r * np.cos(parameter_optimal[1])

x_hs = [x_hs_1, x_hs_2]
y_hs = [y_hs_1, y_hs_2]
z_hs = [z_hs_1, z_hs_2]

HS_main_plot_size = 30
HS_sub_plot_size_1 = m_1_sub_ini / m_1_main_ini * HS_main_plot_size
HS_sub_plot_size_2 = m_2_sub_ini / m_2_main_ini * HS_main_plot_size

ax2.scatter(x_hs[0], y_hs[0], z_hs[0], s = HS_main_plot_size, c = "blue")
ax2.scatter(x_hs[1], y_hs[1], z_hs[1], s = HS_main_plot_size, c = "orange")


######## need to be recode ########
# radiation of belt like bbody
# drawing belt
belt_XYZ, belt_theta, belt_phi = belt_like(N_belt, , phi_belt, theta_main, phi_main)

hs_sub_draw_1 = drawing_sub_hs(parameter_optimal[4], N_sub, parameter_optimal[1], parameter_optimal[2])
print(hs_sub_draw_1)
for i in range(0, N_sub):
    ax2.scatter(hs_sub_draw_1[0, i + 1], hs_sub_draw_1[1, i + 1], hs_sub_draw_1[2, i + 1], zorder = 3, s = HS_sub_plot_size_1, c = "lime")

hs_sub_draw_2 = drawing_sub_hs(parameter_optimal[9], N_sub, parameter_optimal[6], parameter_optimal[7])
for i in range(0, N_sub):
    ax2.scatter(hs_sub_draw_2[0, i + 1], hs_sub_draw_2[1, i + 1], hs_sub_draw_2[2, i + 1], zorder = 3, s = HS_sub_plot_size_2, c = "green")



HS1 = mu(x, parameter_optimal[0], parameter_optimal[1], parameter_optimal[2], parameter_optimal[3], parameter_optimal[4], parameter_optimal[5])
HS2 = mu(x, parameter_optimal[0], parameter_optimal[6], parameter_optimal[7], parameter_optimal[8], parameter_optimal[9], parameter_optimal[10])


######## need to be recode ########
# data of pulse profile
ax1.scatter(x_data, y_data, s = 5, c = "dimgrey")
# fitting of pulse profile
ax1.plot(x, HS1, label = "HS1", color = "blue")
ax1.plot(x, HS2, label = "HS2", color = "orange")
ax1.plot(x, y_fitting, label = 'total', color = 'lime')
# info
ax1.set_xlim(0, 2)
ax1.set_xlabel('Phase')
ax1.set_ylabel('Counts/sec')
ax1.legend(bbox_to_anchor = (1.15, 1), loc = "upper right")
ax1.set_title("SGR 0418+5729, 0.5-10 keV, light-bending model with stationary emission")
# ax.text(1.8, 0.325, r"$\theta = \frac{\pi}{5}$")
# ax.text(1.8, 0.31, r"$i = \frac{\pi}{3}$")
# ax.text(1.8, 0.295, r"$\phi = \pi$")
# ax.text(0.8, 0.32, r"$amp = 0.345$")
# plt.title("3XMM J185246.6+003317のパルス波形と手合わせ曲線", fontname = "MS Gothic")

ax2.set_xlim(-1.1, 1.1)
ax2.set_ylim(-1.1, 1.1)
ax2.set_zlim(-1.1, 1.1)
ax2.view_init(elev = np.degrees(parameter_optimal[0]), azim = 0)
ax2.axis("off")

# ax3.plot(x_data, y_data - func(x_data, a_p_1, b, c_p_1, d_p_1) - func(x_data, a_p_2, b, c_p_2, d_p_2) - func(x_data, a_p_3, b, c_p_3, d_p_3))
ax3.plot(x_data, (y_data - y_fitting_same_size_as_y_data) / y_data)
ax3.axhline(y = 0, linestyle = '--', color = 'grey')
ax3.set_xlim(0, 2)
ax3.set_ylabel('residual/data')
plt.show()
print("inclination angle:", np.degrees(parameter_optimal[0]))
print("Hot_spot_1:", np.degrees(parameter_optimal[1]), np.degrees(parameter_optimal[2]), parameter_optimal[3])
print("Hot_spot_1_sub:", np.degrees(parameter_optimal[4]), parameter_optimal[5])
print("Hot_spot_2:", np.degrees(parameter_optimal[6]), np.degrees(parameter_optimal[7]), parameter_optimal[8])
print("Hot_spot_1_sub:", np.degrees(parameter_optimal[9]), parameter_optimal[10])

SyntaxError: invalid syntax (121697804.py, line 143)